Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')   # 5x5x1x16 conv of stride 2, 0 padding on 28x28x1 to get 14x14x16
    hidden = tf.nn.relu(conv + layer1_biases)                                 # one bias value for each of 16 layers
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME') # 5x5x16x16 conv of stride 2, 0 padding on 14x14x16 to get 7x7x16
    hidden = tf.nn.relu(conv + layer2_biases)                                 # one bias value for each of 16 layers    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])  # flatten to (7x7x16)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)   # multiply by (7x7x16)x64 to get _data_x64
    return tf.matmul(hidden, layer4_weights) + layer4_biases                  # multiply by 64x10 to end up with _data_x10
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.724304
Minibatch accuracy: 18.8%
Validation accuracy: 10.2%
Minibatch loss at step 50: 2.228576
Minibatch accuracy: 31.2%
Validation accuracy: 42.5%
Minibatch loss at step 100: 1.047076
Minibatch accuracy: 62.5%
Validation accuracy: 61.5%
Minibatch loss at step 150: 1.315193
Minibatch accuracy: 56.2%
Validation accuracy: 68.6%
Minibatch loss at step 200: 0.879071
Minibatch accuracy: 81.2%
Validation accuracy: 74.0%
Minibatch loss at step 250: 0.925594
Minibatch accuracy: 87.5%
Validation accuracy: 77.0%
Minibatch loss at step 300: 0.366928
Minibatch accuracy: 93.8%
Validation accuracy: 77.6%
Minibatch loss at step 350: 1.068084
Minibatch accuracy: 75.0%
Validation accuracy: 78.8%
Minibatch loss at step 400: 0.746944
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 450: 0.949299
Minibatch accuracy: 68.8%
Validation accuracy: 80.4%
Minibatch loss at step 500: 0.481578
Minibatch accuracy: 87.5%
Validation accuracy: 81.2%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')   # 5x5x1x16 conv of stride 1, 0 padding on 28x28x1 to get 28x28x16
    hidden = tf.nn.relu(conv + layer1_biases)                                 # one bias value for each of 16 layers
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # max pooling of size 2, stride 2, 0 padding to get 14x14x16 
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME') # 5x5x16x16 conv of stride 1, 0 padding on 14x14x16 to get 14x14x16
    hidden = tf.nn.relu(conv + layer2_biases)                                 # one bias value for each of 16 layers
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # max pooling of size 2, stride 2, 0 padding to get 7x7x16 
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])  # flatten to (7x7x16)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)   # multiply by (7x7x16)x64 to get _data_x64
    return tf.matmul(hidden, layer4_weights) + layer4_biases                  # multiply by 64x10 to end up with _data_x10
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.448282
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.097672
Minibatch accuracy: 25.0%
Validation accuracy: 28.1%
Minibatch loss at step 100: 0.988872
Minibatch accuracy: 62.5%
Validation accuracy: 61.6%
Minibatch loss at step 150: 1.274576
Minibatch accuracy: 56.2%
Validation accuracy: 62.3%
Minibatch loss at step 200: 0.901549
Minibatch accuracy: 81.2%
Validation accuracy: 72.8%
Minibatch loss at step 250: 0.856343
Minibatch accuracy: 81.2%
Validation accuracy: 77.7%
Minibatch loss at step 300: 0.329965
Minibatch accuracy: 93.8%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.847023
Minibatch accuracy: 68.8%
Validation accuracy: 80.0%
Minibatch loss at step 400: 0.808688
Minibatch accuracy: 87.5%
Validation accuracy: 80.4%
Minibatch loss at step 450: 0.835783
Minibatch accuracy: 68.8%
Validation accuracy: 81.0%
Minibatch loss at step 500: 0.452561
Minibatch accuracy: 81.2%
Validation accuracy: 82.1%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [9]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob = tf.placeholder(tf.float32)
    
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')   # 5x5x1x16 conv of stride 1, 0 padding on 28x28x1 to get 28x28x16
    hidden = tf.nn.relu(conv + layer1_biases)                                 # one bias value for each of 16 layers
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # max pooling of size 2, stride 2, 0 padding to get 14x14x16 
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME') # 5x5x16x16 conv of stride 1, 0 padding on 14x14x16 to get 14x14x16
    hidden = tf.nn.relu(conv + layer2_biases)                                 # one bias value for each of 16 layers
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # max pooling of size 2, stride 2, 0 padding to get 7x7x16 
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])  # flatten to (7x7x16)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)   # multiply by (7x7x16)x64 to get _data_x64
    hidden = tf.nn.dropout(hidden, keep_prob)                                 # add dropout
    return tf.matmul(hidden, layer4_weights) + layer4_biases                  # multiply by 64x10 to end up with _data_x10
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.75}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 3.610254
Minibatch accuracy: 18.8%
Validation accuracy: 11.7%
Minibatch loss at step 50: 2.379420
Minibatch accuracy: 0.0%
Validation accuracy: 10.9%
Minibatch loss at step 100: 2.223381
Minibatch accuracy: 6.2%
Validation accuracy: 20.1%
Minibatch loss at step 150: 1.947127
Minibatch accuracy: 25.0%
Validation accuracy: 25.9%
Minibatch loss at step 200: 1.396066
Minibatch accuracy: 50.0%
Validation accuracy: 57.0%
Minibatch loss at step 250: 1.454372
Minibatch accuracy: 56.2%
Validation accuracy: 69.1%
Minibatch loss at step 300: 0.636408
Minibatch accuracy: 75.0%
Validation accuracy: 74.3%
Minibatch loss at step 350: 1.320347
Minibatch accuracy: 56.2%
Validation accuracy: 77.1%
Minibatch loss at step 400: 1.087924
Minibatch accuracy: 56.2%
Validation accuracy: 76.2%
Minibatch loss at step 450: 0.977536
Minibatch accuracy: 56.2%
Validation accuracy: 78.8%
Minibatch loss at step 500: 0.710755
Minibatch accuracy: 68.8%
Validation accuracy: 79.5%
Min